In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
# 차트에서 한글 출력을 위한 설정
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

# 고객 당 한줄로 표현해야 feature로 쓸 수 있다.!!!!!!!!!!

In [3]:
tr = pd.read_csv('./data/train_transactions.csv', encoding='cp949') # ../ 에서 ./로 바꿈

In [4]:
tr

,cust_id,tran_date,store_nm,goods_id,gds_grp_nm,gds_grp_mclas_nm,amount
0,0,2007-01-19 00:00:00,강남점,127105,기초 화장품,화장품,850000
1,0,2007-03-30 00:00:00,강남점,342220,니 트,시티웨어,480000
2,0,2007-03-30 00:00:00,강남점,127105,기초 화장품,화장품,3000000
3,0,2007-03-30 00:00:00,강남점,342205,니 트,시티웨어,840000
4,0,2007-03-30 00:00:00,강남점,342220,상품군미지정,기타,20000
...,...,...,...,...,...,...,...
231999,3499,2007-12-17 00:00:00,본 점,127129,상품군미지정,기타,-135000
232000,3499,2007-12-23 00:00:00,노원점,285136,시티웨어,시티웨어,6380000
232001,3499,2007-12-23 00:00:00,노원점,39107,야채,농산물,40800
232002,3499,2007-12-27 00:00:00,본 점,740120,어덜트,명품,4880000


In [6]:
# tr_test = pd.read_csv('../data/test_transactions.csv', encoding='cp949')

FileNotFoundError: [Errno 2] No such file or directory: './data/test_transactions.csv'

In [3]:
# y_train = pd.read_csv('../data/y_train.csv').gender
# IDtest = tr_test.cust_id.unique()

# tr_train.head()

# Feature Generation
새로 만든 feature와 병합할 고객 ID로만 이루어진 데이터프레임을 만든다.

In [7]:
features = pd.DataFrame({'cust_id':tr.cust_id.unique()})

In [8]:
features

,cust_id
0,0
1,1
2,2
3,3
4,4
...,...
3495,3495
3496,3496
3497,3497
3498,3498


- unique() 처리 이후 232004 rows --> 3500 rows

# Numeric Features

[파생변수 1-4] 총구매액, 구매건수, 평균구매액, 최대구매액

- `df.groupby(["~를"])["~로"]` --> 이렇게 활용하면 된다!

- `agg함수` --> 여러가지 계산 가능! 내가 원하는 다양한 수치를 얻을 수 있다!

- `set_index()` --> 행번호 인덱스가 아닌 컬럼명을 지정한 인덱스가 생성
- `reset_index()` --> 컬럼명 인덱스가 아닌 행 번호 인덱스로 사용하고 싶은 경우..에도 사용 가능!

https://ponyozzang.tistory.com/617

In [11]:
a = tr.groupby('cust_id')['amount'].agg([('총구매액', np.sum), ('구매건수', np.size), ('평균구매액', lambda x: np.round(np.mean(x)))])

In [12]:
a

,총구매액,구매건수,평균구매액
cust_id,,,
0,68282840,74,922741.0
1,2136000,3,712000.0
2,3197000,4,799250.0
3,16077620,44,365400.0
4,29050000,3,9683333.0
...,...,...,...
3495,3175200,2,1587600.0
3496,29628600,13,2279123.0
3497,75000,1,75000.0


In [13]:
a.reset_index()

,cust_id,총구매액,구매건수,평균구매액
0,0,68282840,74,922741.0
1,1,2136000,3,712000.0
2,2,3197000,4,799250.0
3,3,16077620,44,365400.0
4,4,29050000,3,9683333.0
...,...,...,...,...
3495,3495,3175200,2,1587600.0
3496,3496,29628600,13,2279123.0
3497,3497,75000,1,75000.0
3498,3498,1875000,2,937500.0


In [14]:
f = tr.groupby('cust_id')['amount'].agg([
    ('총구매액',np.sum), 
    ('구매건수', np.size), 
    ('평균구매액', lambda x: np.round(np.mean(x))),
    ('최대구매액', np.max),
]).reset_index()

In [15]:
f

,cust_id,총구매액,구매건수,평균구매액,최대구매액
0,0,68282840,74,922741.0,11264000
1,1,2136000,3,712000.0,2136000
2,2,3197000,4,799250.0,1639000
3,3,16077620,44,365400.0,4935000
4,4,29050000,3,9683333.0,24000000
...,...,...,...,...,...
3495,3495,3175200,2,1587600.0,3042900
3496,3496,29628600,13,2279123.0,7200000
3497,3497,75000,1,75000.0,75000
3498,3498,1875000,2,937500.0,1000000


In [18]:
features = features.merge(f, how='left'); features # 여러줄 쓸때 이렇게 쓰나보다..

,cust_id,총구매액,구매건수,평균구매액,최대구매액
0,0,68282840,74,922741.0,11264000
1,1,2136000,3,712000.0,2136000
2,2,3197000,4,799250.0,1639000
3,3,16077620,44,365400.0,4935000
4,4,29050000,3,9683333.0,24000000
...,...,...,...,...,...
3495,3495,3175200,2,1587600.0,3042900
3496,3496,29628600,13,2279123.0,7200000
3497,3497,75000,1,75000.0,75000
3498,3498,1875000,2,937500.0,1000000


,cust_id,총구매액,구매건수,평균구매액,최대구매액
0,0,68282840,74,922741.0,11264000
1,1,2136000,3,712000.0,2136000
2,2,3197000,4,799250.0,1639000
3,3,16077620,44,365400.0,4935000
4,4,29050000,3,9683333.0,24000000
...,...,...,...,...,...
3495,3495,3175200,2,1587600.0,3042900
3496,3496,29628600,13,2279123.0,7200000
3497,3497,75000,1,75000.0,75000
3498,3498,1875000,2,937500.0,1000000
